In [14]:
import pandas as pd

In [15]:
all_data = pd.read_csv("ljic219b15.20_.ismr", header=None, squeeze=False)
all_data

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,2117,350160,10,630,3.0,26.0,41.8,0.097,0.082,0.035,...,0.028,-9.506,0.127,2850,0.017,1.593,1.53,0.000147,0.000102,0.000061
1,2117,350160,12,630,61.0,14.0,40.5,0.147,0.094,0.032,...,nan,nan,nan,nan,nan,nan,nan,0.000128,0.000536,nan
2,2117,350160,18,630,169.0,64.0,49.0,0.043,0.036,0.013,...,0.017,-26.416,0.029,13210,0.006,0.609,1.51,0.000019,0.000018,0.000006
3,2117,350160,20,630,38.0,49.0,47.2,0.040,0.044,0.015,...,nan,nan,nan,nan,nan,nan,nan,0.000030,nan,nan
4,2117,350160,21,630,304.0,36.0,45.9,0.045,0.051,0.017,...,nan,nan,nan,nan,nan,nan,nan,0.000033,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2117,351000,162,630,46.0,28.0,43.9,0.105,0.064,0.021,...,nan,nan,nan,nan,nan,nan,nan,0.000058,nan,nan
493,2117,351000,164,630,223.0,27.0,45.6,0.107,0.052,0.018,...,nan,nan,nan,nan,nan,nan,nan,0.000048,nan,nan
494,2117,351000,166,630,269.0,71.0,52.7,0.025,0.023,0.013,...,nan,nan,nan,nan,nan,nan,nan,0.000021,nan,nan
495,2117,351000,174,630,274.0,11.0,38.6,0.163,0.118,0.038,...,0.034,-5.839,0.153,1819,0.030,2.686,1.45,0.000153,nan,0.000075


In [17]:
m=list(all_data[7])
m[0]

0.09699999999999999

In [3]:
data = pd.read_csv("ljic219b15.20_.ismr", header=None, usecols=[0,1,2,7,4,5], squeeze=True)
WN = data[0]
TOW = data[1]
PRN = data[2]
S4 = data[7] # S4 in "Sig1"
alpha = data[4]
epsilon = data[5]

alpha

0        3.0
1       61.0
2      169.0
3       38.0
4      304.0
       ...  
492     46.0
493    223.0
494    269.0
495    274.0
496    160.0
Name: 4, Length: 497, dtype: float64

In [4]:
WN

0      2117
1      2117
2      2117
3      2117
4      2117
       ... 
492    2117
493    2117
494    2117
495    2117
496    2117
Name: 0, Length: 497, dtype: int64

In [5]:
TOW

0      350160
1      350160
2      350160
3      350160
4      350160
        ...  
492    351000
493    351000
494    351000
495    351000
496    351000
Name: 1, Length: 497, dtype: int64

In [6]:
# GPS time: week & seconds; to UTC time.
# OJO: It's missing the leapseconds, these value get from the navigation file
def WeekSeconds2UTC(gpsweek,gpsseconds,leapseconds):
    import datetime
    
    datetimeformat = "%Y-%m-%d %H:%M:%S"
    first_epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
    elapsed = datetime.timedelta(days=(gpsweek*7),seconds=(gpsseconds-leapseconds))
    
    # LISN date format: 2 digit year, day of year, seconds since midnight for a day 
    date = datetime.datetime.strftime(first_epoch + elapsed,"%y-%j;%H:%M:%S")
    
    year = int(date[:2])
    day = int(date[3:6])
    hour = int(date[7:9])
    minute = int(date[10:12])
    second = int(date[13:15])
    seconds_day = second*1 + minute*60 + hour*60*60
    
    return {"year":year, "day_year":day, "seconds_day": seconds_day}
    

In [7]:
a = WeekSeconds2UTC(2119,342000,0)

In [8]:
a["year"], a["day_year"], a["seconds_day"]

(20, 232, 82800)

In [9]:
# The PRNs were obtained from PolaRx5S Reference Guide
def get_PRN(svid):
    if 1<=svid<=37:
        prn = "G"+str(svid)
    elif 38<=svid<=61:
        prn = "R"+str(svid-37)
    elif svid==62:
        prn = "NA"
    elif 63<=svid<=68:
        prn = "R"+str(svid-38)
    elif 71<=svid<=106:
        prn = "E"+str(svid-70)
    elif 107<=svid<=119:
        prn = "NA"
    elif 120<=svid<=140:
        prn = "S"+str(svid-100)
    elif 141<=svid<=177:
        prn = "C"+str(svid-140)
    elif 181<=svid<=187:
        prn = "J"+str(svid-180)
    elif 191<=svid<=197:
        prn = "I"+str(svid-190)
    elif 198<=svid<=215:
        prn = "S"+str(svid-157)
    elif 216<=svid<=222:
        prn = "I"+str(svid-208)
    else:
        prn = "svid not valid!"
    
    return prn

In [10]:
test = get_PRN(49)
test

'R12'

In [11]:
PRN[0]

10

In [12]:
yr = [] # year
dyr = [] # day_year
sd = [] # seconds_day
nd = [1]*len(WN)
prn = [] # prn
for i in range(len(WN)):
    fecha = WeekSeconds2UTC(int(WN[i]), int(TOW[i]), 0)
    yr.append(fecha["year"])
    dyr.append(fecha["day_year"])
    sd.append(fecha["seconds_day"])
    
    prn.append(get_PRN(int(PRN[i])))

In [13]:
list_tuples = list(zip(yr,dyr,sd,nd,prn)) # creates a list of tuples
dataFrame = pd.DataFrame(list_tuples, columns=['year','day','seconds','Ndata','PRN']) # creates a pandas dataframe
# Adding other pandas series
#dataFrame = pd.concat([dataFrame, alpha.to_frame()],ignore_index=True)
#dataFrame = dataFrame.append([alpha], ignore_index=True)
dataFrame['S4'] = S4
dataFrame['alpha'] = alpha
dataFrame['epsilon'] = epsilon
#alpha
dataFrame

,year,day,seconds,Ndata,PRN,S4,alpha,epsilon
0,20,219,4560,1,G10,0.097,3.0,26.0
1,20,219,4560,1,G12,0.147,61.0,14.0
2,20,219,4560,1,G18,0.043,169.0,64.0
3,20,219,4560,1,G20,0.040,38.0,49.0
4,20,219,4560,1,G21,0.045,304.0,36.0
...,...,...,...,...,...,...,...,...
492,20,219,5400,1,C22,0.105,46.0,28.0
493,20,219,5400,1,C24,0.107,223.0,27.0
494,20,219,5400,1,C26,0.025,269.0,71.0
495,20,219,5400,1,C34,0.163,274.0,11.0
